In [1]:
from google.cloud import pubsub_v1

In [7]:
credential_json_path = "../handy-bindery-401205-33cea1d16721.json"

In [8]:
### publisher_client 생성 
publisher_client = pubsub_v1.PublisherClient.from_service_account_json(credential_json_path)
publisher_client

In [21]:
### 토픽 생성 
proj_id = "handy-bindery-401205"
topic_name= "food_topic"
topic_path = publisher_client.topic_path(proj_id, topic_name)

In [22]:
topic_path

'projects/handy-bindery-401205/topics/food_topic'

In [23]:
### 토픽 리스트 확인 
publisher_client.list_topics(request = {"project":f"projects/{proj_id}"})

ListTopicsPager<topics {
  name: "projects/handy-bindery-401205/topics/food_topic"
}
>

### 해당 topic에 데이터 전송

In [25]:
import time

In [46]:
import datetime

In [48]:
datetime.datetime.now()

datetime.datetime(2023, 11, 13, 18, 17, 42, 75679)

In [1]:
data_csv_path = "../Batch_pipeline_dir/food_daily.csv"
with open(data_csv_path) as csv_file:
    for row in csv_file:
#         data = row.encode('utf-8-sig')
#         publish_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        data = row.encode('utf-8-sig')

        ### topic에 data -> publish
        future = publisher_client.publish(topic_path, data=data)
#         print(future.result())
        time.sleep(1)


### 위에서 해당 topic으로 전송 잘 되는것을 확인 가능

#### subscriber_client 생성

In [28]:
topic_path

'projects/handy-bindery-401205/topics/food_topic'

In [29]:
subscriber_client = pubsub_v1.SubscriberClient.from_service_account_json(credential_json_path)
subscriber_client

In [32]:
### subscription_path 설정 
for subscription in subscriber_client.list_subscriptions(request = {"project":f"projects/{proj_id}"}):
    print(f"subscription name: {subscription.name}")

subscription name: projects/handy-bindery-401205/subscriptions/data_from_file_sub
subscription name: projects/handy-bindery-401205/subscriptions/food_topic_sub


In [33]:
subscription_id = "food_topic_sub"
subscription_path = subscriber_client.subscription_path(proj_id, subscription_id)
subscription_path

'projects/handy-bindery-401205/subscriptions/food_topic_sub'

In [36]:
### 메세지를 전달받을 Callback 함수 정의 
def callback(message):
    print(F"Received MEssage: {message.data.decode('utf-8-sig')}")
#     print(message)
    ## 메세지 처리후 확인 메세지 보내기
    message.ack()

In [39]:
### 구독을 생성하고 Callback 함수 연결합니다 
streaming_pull_future = subscriber_client.subscribe(subscription_path,callback=callback)
timeout = 100


In [41]:
# streaming_pull_future.result(timeout=timeout)

#### subscriber.py를 따로 작성해서 추후에 테스트

### 이제 streaming beam pipeline으로 bigQuery에 적재 하는 파이프라인 구성


### bigquery client로 적재될 bigquery dataset, table 설정

In [55]:
from google.cloud import bigquery

In [58]:
import apache_beam as beam

In [59]:
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions

In [60]:

#project-id:dataset_id.table_id
delivered_table_spec = f'{proj_id}:{dataset_id}.delivered_orders'
#project-id:dataset_id.table_id
other_table_spec = f'{proj_id}:{dataset_id}.other_status_orders'


In [61]:
## bigquery_client 
bq_client = bigquery.Client.from_service_account_json(credential_json_path)
bq_client

In [70]:
## BigQuery 내의 dataset list 확인
for obj in bq_client.list_datasets():
    print(obj.dataset_id)

bq_beam_dataset
bq_test_dataset
google_public_dataset


In [74]:
bq_client.project

'handy-bindery-401205'

In [77]:
dataset_id = f"{bq_client.project}.bq_pubsub_beam_dataset"
dataset_id

'handy-bindery-401205.bq_pubsub_beam_dataset'

In [76]:

dataset=bigquery.Dataset(dataset_id)
dataset.location = 'US'
dataset.description = 'pubsub beam dataset streaming orders'
### dataset 생성:'handy-bindery-401205.bq_pubsub_beam_dataset'
dataset_ref = bq_client.create_dataset(dataset, timeout=30)

In [78]:
## BigQuery 내의 dataset list 확인
for obj in bq_client.list_datasets():
    print(obj.dataset_id)

bq_beam_dataset
bq_pubsub_beam_dataset
bq_test_dataset
google_public_dataset


### table Schema 설정

In [79]:
table_schema = 'customer_id:STRING,date:STRING,timestamp:STRING,order_id:STRING,items:STRING,amount:STRING,mode:STRING,restaurant:STRING,status:STRING,ratings:STRING,feedback:STRING,new_col:STRING'


#### 이후에는 python script로 다시 작성